In [1]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option("display.max_columns", 100)

### Load raw dataset

In [3]:
RAW_PATH = "../data/raw/BDIR61FL.csv"

df_raw = pd.read_csv(RAW_PATH)  # df_raw = OG copy
df = df_raw.copy()              # df = working copy

### Shape 

In [4]:
df.shape


(17842, 1827)

In [5]:
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17842 entries, 0 to 17841
Columns: 1827 entries, caseid to s541y_3
dtypes: float64(1599), int64(224), object(4)
memory usage: 248.7+ MB


### Null Columns 

In [6]:
null_cols = df.columns[df.isnull().all()]
len(null_cols)


0

### Columns with only ONE unique value

In [7]:
constant_cols = df.columns[df.nunique() <= 1]
len(constant_cols)


390

In [8]:
df.drop(columns=constant_cols, inplace=True)


### Identity Info drop

In [9]:
id_like_cols = [
    "caseid", "hhid", "cluster", "line", "v001", "v002", "v003", "v004", "v005", "v006", "v007","v008","v009"
]

df.drop(columns=id_like_cols, errors="ignore", inplace=True)


### Checkpoint_1

In [11]:
CLEAN_PATH = "../data/processed/01_basic_cleaned.csv"
df.to_csv(CLEAN_PATH, index=False)


KeyboardInterrupt: 

### Sanity check

In [10]:
print("Before:", df_raw.shape)
print("After :", df.shape)


Before: (17842, 1827)
After : (17842, 1428)


## PHASE 2: Feature Decoding and Interpretation


### Inspect column names

In [8]:
df.columns[:50]


Index(['v000', 'v010', 'v011', 'v012', 'v013', 'v014', 'v015', 'v016', 'v017',
       'v018', 'v019', 'v019a', 'v020', 'v021', 'v022', 'v023', 'v024', 'v025',
       'v026', 'v027', 'v028', 'v029', 'v030', 'v031', 'v032', 'v034', 'v040',
       'v042', 'v101', 'v102', 'v106', 'v107', 'v113', 'v115', 'v116', 'v119',
       'v120', 'v121', 'v122', 'v123', 'v124', 'v127', 'v128', 'v129', 'v130',
       'v133', 'v134', 'v135', 'v136', 'v137'],
      dtype='object')

### Search anima variables 

In [10]:
[col for col in df.columns if "hb" in col.lower() or "anem" in col.lower()]


['shb1_1',
 'shb1_2',
 'shb1_3',
 'shb1_4',
 'shb1d_1',
 'shb1d_2',
 'shb1d_3',
 'shb1d_4',
 'shb1m_1',
 'shb1m_2',
 'shb1m_3',
 'shb1m_4',
 'shb1y_1',
 'shb1y_2',
 'shb1y_3',
 'shb1y_4',
 'shb2_1',
 'shb2_2',
 'shb2_3',
 'shb2_4',
 'shb2d_1',
 'shb2d_2',
 'shb2d_3',
 'shb2d_4',
 'shb2m_1',
 'shb2m_2',
 'shb2m_3',
 'shb2m_4',
 'shb2y_1',
 'shb2y_2',
 'shb2y_3',
 'shb2y_4',
 'shb3_1',
 'shb3_2',
 'shb3_3',
 'shb3_4',
 'shb3d_1',
 'shb3d_2',
 'shb3d_3',
 'shb3d_4',
 'shb3m_1',
 'shb3m_2',
 'shb3m_3',
 'shb3m_4',
 'shb3y_1',
 'shb3y_2',
 'shb3y_3',
 'shb3y_4']

### Target Variable


Hemoglobin Level	V453 / HA53	                  
Anemia Status	V457 / HA57	Anemia level                     
we will name 'hb' for level and 'anemia' for binary 

In [11]:
df["hb"] = df["v453"].replace(999, pd.NA)


In [13]:
df["hb"].describe()


count     5727.0
unique     104.0
top        123.0
freq       191.0
Name: hb, dtype: float64

### Create binary anemia label

In [12]:
df["anemia"] = (df["hb"] < 12.0).astype(int) #non-pregnant women


In [15]:
df["anemia"] = np.where(                                #pregnant women       
    (df["pregnant"] == 1) & (df["hb"] < 11.0), 1,
    np.where(df["hb"] < 12.0, 1, 0)
) 

KeyError: 'pregnant'

### compare v and h

In [15]:
df[["hb", "v457"]].dropna().head()


,hb,v457
5,11.0,3.0
9,13.0,4.0
17,10.4,3.0
20,13.6,4.0
26,11.8,3.0


### fix g/dl

In [14]:
df["hb"] = df["hb"].replace(994, pd.NA) / 10


## Phase 2.6 to 2.8 Feature Selection  and Encoding  

https://docs.google.com/spreadsheets/d/1OCGH5YABR8Lu_VlW97hbYaG_agCxWnVgf20BlLgSswU/edit?gid=2052048461#gid=2052048461

### Panic button

In [16]:
df_full = df.copy()


### DROP COLUMNS

In [20]:

keep_cols = [
    "hb",          # target source 
    # --- A. Demographic Characteristics ---
    'v012', # Age
    'v501', # Marital Status
    'v025', # Residence (Urban/Rural)
    'v024', # Region
    'v136', # Household Size
    'v511', # Age at 1st Marriage
    'v218', # Number of Living Children
    'v222', # Birth Interval
    'v228', # Terminated pregnancy

    # --- B. Socio-Economic Status ---
    'v106', # Education Level
    'v133', # Years of Schooling
    'v190', # Wealth Index
    'v714', # Employment Status
    'v716', # Occupation
    'v119', # Electricity
    'v127', # Main Floor Material
    'v129', # Main Roof Material
    'v128', # Main Wall Material
    'v121', # Has Television
    'v153', # Has Telephone/Mobile (Check if v169a is better)
    
    # --- C. Lifestyle & Behavioral Factors ---
    'v161', # Cooking Fuel
    
    'v157', # Frequency reading newspaper
    'v158', # Frequency listening to radio
    'v159', # Frequency watching TV
    'v312', # Current Contraceptive Method

    # --- D. Environmental & Sanitation ---
    'v113', # Source of Drinking Water
    'v116', # Type of Toilet Facility
    'v160', # Shared Toilet
    'v115', # Time to get to Water Source
    'v040', # Cluster Altitude (meters)

    # --- E. Health & Nutrition Indicators ---
    'v445', # Body Mass Index (BMI)
    'v213', # Pregnancy Status
    'v404', # Currently Breastfeeding
    'v215', # Time since last menstrual period
]



### Subset df

In [22]:
df = df[keep_cols]


### Sanity Check

In [4]:
df.shape
df.head()
df.isnull().sum().sort_values(ascending=False)


occupation              15149
hb                      12137
birth_interval           1817
shared_toilet             606
age                         0
marital_status              0
household_size              0
age_first_marriage          0
living_children             0
terminated_pregnancy        0
residence                   0
education_level             0
years_schooling             0
wealth_index                0
employment_status           0
has_electricity             0
floor_material              0
roof_material               0
region                      0
wall_material               0
has_tv                      0
cooking_fuel                0
has_telephone               0
freq_radio                  0
freq_tv                     0
contraceptive_method        0
freq_newspaper              0
water_source                0
toilet_type                 0
time_to_water               0
altitude                    0
bmi                         0
is_pregnant                 0
is_breastf

### Rename Col

In [26]:
df.rename(columns={
    # --- A. Demographic Characteristics ---
    "v012": "age",
    "v501": "marital_status",
    "v025": "residence",
    "v024": "region",
    "v136": "household_size",
    "v511": "age_first_marriage",
    "v218": "living_children",
    "v222": "birth_interval",
    "v228": "terminated_pregnancy",

    # --- B. Socio-Economic Status ---
    "v106": "education_level",
    "v133": "years_schooling",
    "v190": "wealth_index",
    "v714": "employment_status",
    "v716": "occupation",
    "v119": "has_electricity",
    "v127": "floor_material",
    "v129": "roof_material",
    "v128": "wall_material",
    "v121": "has_tv",
    "v153": "has_telephone",       # Note: v153 is often landline. Check if v169a (mobile) is available/better.

    # --- C. Lifestyle & Behavioral Factors ---
    "v161": "cooking_fuel",
    "v157": "freq_newspaper",
    "v158": "freq_radio",
    "v159": "freq_tv",
    "v312": "contraceptive_method",

    # --- D. Environmental & Sanitation ---
    "v113": "water_source",
    "v116": "toilet_type",
    "v160": "shared_toilet",
    "v115": "time_to_water",
    "v040": "altitude",

    # --- E. Health & Nutrition Indicators ---
    "v445": "bmi",
    "v213": "is_pregnant",
    "v404": "is_breastfeeding",
    "v215": "time_since_period",
    
    # --- Target Variables (If you haven't renamed them yet) ---
    "v453": "hg",
    
}, inplace=True)

### Checkpoint 2

In [27]:
df.to_csv("../data/processed/feature_reduced.csv", index=False)


In [3]:
PROC_PATH = "../data/processed/feature_reduced.csv"

df = pd.read_csv(PROC_PATH)             # df = working copy

In [5]:
df.shape


(17842, 35)

### Remove Noise or redundancy 

In [6]:
cols_to_drop = [
    "iron_supplementation",
    "birth_interval",
    "terminated_pregnancy",
    "time_since_period",
    "years_schooling"
]


In [7]:
df.drop(columns=cols_to_drop, inplace=True, errors="ignore")


### Sanity check

In [15]:
df.columns
df.shape


(17842, 31)

### Checkpoint 3

In [9]:
df.to_csv("../data/processed/feature_reduced_2.csv", index=False)


### RollBack

In [10]:
df_before_missing = df.copy()


### Count Missing value per Col

In [11]:
missing_counts = df.isna().sum().sort_values(ascending=False)
missing_percent = (missing_counts / len(df)) * 100

missing_summary = pd.DataFrame({
    "missing_count": missing_counts,
    "missing_percent": missing_percent
})

missing_summary


,missing_count,missing_percent
occupation,15149,84.906401
hb,12137,68.024885
shared_toilet,606,3.396480
residence,0,0.000000
region,0,0.000000
household_size,0,0.000000
age,0,0.000000
age_first_marriage,0,0.000000
living_children,0,0.000000
wealth_index,0,0.000000


In [ ]:
cols_to_drop = [
    "occupation",
]

In [12]:
df.drop(columns=cols_to_drop, inplace=True, errors="ignore")


### Drop rows with missing target  

In [16]:
df.columns
df.shape

(17842, 31)

In [17]:
df = df.dropna(subset=["hb"])


In [18]:
df.columns
df.shape

(5705, 31)

### CREATE BINARY LABEL

In [19]:
df["anemia"] = (df["hb"] < 12.0).astype(int)


C:\Users\Mahir\AppData\Local\Temp\ipykernel_3864\3435557492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["anemia"] = (df["hb"] < 12.0).astype(int)


### Checkpoint 4

In [20]:
df.to_csv("../data/processed/feature_reduced_3.csv", index=False)


In [ ]:
# phase 4 STEP 5.1. (start from here tomorrow)

In [3]:
PROC_PATH = "../data/processed/feature_reduced_3.csv"

df = pd.read_csv(PROC_PATH)             # df = working copy

In [8]:
df.columns
df.shape

(5705, 31)

In [5]:
cols_to_drop = [
    "occupation",
]

In [6]:
df = df.drop(columns=cols_to_drop)

### Checkpoint 4

In [7]:
df.to_csv("../data/processed/processed_4.csv", index=False)
